In [16]:
import os
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web
from collections import namedtuple
from operator import itemgetter

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
import streamlit as st

from load_data import *

pd.options.plotting.backend = "plotly"


In [5]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

In [25]:
df = pd.concat([rf_rates['Adj Close'], SPY_df['Close']], axis=1, join='inner')
df.rename(columns={'Adj Close': 'DTB3', 'Close': 'SPY'}, inplace=True)
df['Return'] = np.log1p(df['SPY'].pct_change())
df.drop('SPY', axis=1, inplace=True)
df['Return'] = df['Return'].rolling(window=60).mean()
df['DTB3'] = df['DTB3'].rolling(window=60).mean()
df.head()

,DTB3,Return
Date,,
1960-01-04,NaN,NaN
1960-01-05,NaN,NaN
1960-01-06,NaN,NaN
1960-01-07,NaN,NaN
1960-01-08,NaN,NaN


In [26]:
df.corr()

,DTB3,Return
DTB3,1.000000,-0.051168
Return,-0.051168,1.000000


In [8]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
current = table[0]
hist = table[1]


In [14]:
added = hist['Date'].join(hist['Added'])
added['inSPY'] = True
added['Date'] = pd.to_datetime(added['Date'])
added

,Date,Ticker,Security,inSPY
0,2023-01-05,NaN,NaN,True
1,2023-01-04,GEHC,GE HealthCare,True
2,2022-12-22,STLD,Steel Dynamics,True
3,2022-12-19,FSLR,First Solar,True
4,2022-12-19,NaN,NaN,True
...,...,...,...,...
315,1999-06-09,WLP,Wellpoint,True
316,1998-12-11,FSR,Firstar,True
317,1998-12-11,CCL,Carnival Corp.,True
318,1998-12-11,CPWR,Compuware,True


In [16]:
removed = hist['Date'].join(hist['Removed'])
removed['inSPY'] = False
removed['Date'] = pd.to_datetime(removed['Date'])
removed


,Date,Ticker,Security,inSPY
0,2023-01-05,VNO,Vornado Realty Trust,False
1,2023-01-04,NaN,NaN,False
2,2022-12-22,ABMD,Abiomed,False
3,2022-12-19,FBHS,Fortune Brands Home & Security,False
4,2022-12-19,MBC,MasterBrand,False
...,...,...,...,...
315,1999-06-09,HPH,Harnischfeger Industries,False
316,1998-12-11,LDW,Amoco,False
317,1998-12-11,GRN,General Re,False
318,1998-12-11,SUN,SunAmerica,False


In [33]:
hist = pd.concat([added, removed]).sort_values('Date', ascending=False)
hist.dropna(inplace=True)
hist.Ticker.value_counts()

OI      3
CBE     3
MXIM    3
UA      3
COV     3
       ..
MGM     1
PKG     1
BBBY    1
MUR     1
USL     1
Name: Ticker, Length: 513, dtype: int64

In [26]:
import sys

In [28]:
print(sys.getsizeof(hist.index.values), sys.getsizeof(hist.index.to_list()))

5224 5176


In [6]:
df1 = pd.read_csv('data/SPY-Info.csv')
df2 = pd.read_csv('data/SPY-Historical.csv', index_col='Date')    

In [7]:
df2

,Ticker,Security,inSPY
Date,,,
2023-01-05,VNO,Vornado Realty Trust,False
2023-01-04,GEHC,GE HealthCare,True
2022-12-22,ABMD,Abiomed,False
2022-12-22,STLD,Steel Dynamics,True
2022-12-19,FBHS,Fortune Brands Home & Security,False
...,...,...,...
1998-12-11,CPWR,Compuware,True
1998-12-11,CCL,Carnival Corp.,True
1998-12-11,FSR,Firstar,True


In [13]:
SPY_info_df = get_SPY_info()
# dict(SPY_info_df['Sub-Industry'].value_counts())
SPY_info_df.head()

,Security,SEC filings,Sector,Sub-Industry,Headquarters Location,Date added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [54]:
def url_get_contents(url):
    '''Opens website and reads the binary contents (HTTP Response Body)'''

    req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    f = urlopen(req)
    
    return f.read() # reading contents of the website
    
def get_russell_2000_info():
    '''Makes dataframe of Russell 2000 stock names and tickers'''

    url = 'https://bullishbears.com/russell-2000-stocks-list/'
    xhtml = url_get_contents(url).decode('utf-8')
    p = HTMLTableParser() # Defining the HTMLTableParser object
    p.feed(xhtml) # feeding the html contents in the HTMLTableParser object  
    d = {}

    for i, x in enumerate(p.tables[:-3], 1):
        # print(x[0], x[-1])
        if i % 2:
            d.setdefault('Ticker', []).extend(x)
        else:
            d.setdefault('Security', []).extend(x)
    
    d['Ticker'] = [x[0] for x in d['Ticker'] if x[0] != '']
    d['Security'] = [x[0] for x in d['Security'] if x[0] != '']

    df = pd.DataFrame.from_dict(d, orient='columns')
    df.to_csv('data/Russell 2000 Info.csv')
    
    return df


In [59]:
rus2000_info_df = get_russell_2000_info()

In [88]:
# rus2000_tickers = rus2000_info_df.Ticker.to_list()
path = 'data/market_data/russell_2000'
ndwnld = len(os.listdir(path))
n = len(rus2000_info_df)
not_downloaded = []
folder = os.listdir(path)
to_download = set(rus2000_info_df.Ticker.to_list()) - set([x.replace('.csv', '') for x in folder])
nto_download = len(to_download)
# RUT_df = si.get_data('^RUT')
# RUT_df.to_csv('data/market_data/RUT.csv')
print(f'{len(os.listdir(path))}')

for i, ticker in enumerate(to_download, 1):
    fname = f'{ticker}.csv'
    # if fname not in folder:
    try:
        df = si.get_data(ticker)
        df.to_csv(os.path.join(path, fname))
        print(f"\r{i + 1}/{nto_download} ({(i + 1) / nto_download:.2%}) scanned -", 
                f"{len(os.listdir(path))}/{n} ({(len(os.listdir(path))) / n:.2%})",
                "of Russell 2000 market data downloaded",
                end='', flush=True)
    except Exception as e:
        not_downloaded.append(ticker)

print(f'\r{len(not_downloaded)} not downloaded'.ljust(100, ' '))

1610
389 not downloaded                                                                                 


In [98]:
SPY_df = get_SPY_data()
end = SPY_df.iloc[-1].name.date()
n = 1
start = end - timedelta(days=365 * n)
print(f'start: {start}, end: {end}')

start: 2022-01-06, end: 2023-01-06


In [33]:
with open('data/financial_statements.pickle', 'rb') as f:
    statements = pickle.load(f)

statements['FB'].index.values

array(['Cash and Short Term Investments', 'Receivables',
       'Other current assets', 'Total current assets',
       'Property, Plant, Equpment (Net)', 'Long-Term Investments',
       'Goodwill and Intangible Assets (Total)',
       'Long-term assets (Other)', 'Total non-current assets',
       'Total Assets', 'Accounts Payable', 'Accrued Expenses',
       'Current Part of Debt', 'Current Part of Capital Lease',
       'Current Revenue (Deferred)', 'Other current liabilities',
       'Total current liabilities', 'Long Term Debt (Total)',
       'Long Term Tax Liability (Deferred)',
       'Non-current Liabilities (Other)', 'Total non-current liabilities',
       'Total liabilities', 'Additional Paid In Capital',
       'Common Stock (Net)', 'Retained Earnings', 'Common Equity (Total)',
       'Shareholders Equity (Total)',
       'Shareholders Equity and Liabilities (Total)', 'Shares (Common)',
       'Shareholders Equity (Tangible)', 'Net Debt', 'Total Debt',
       'Revenue', 'Reve

In [9]:
SPY_info_df = get_SPY_info()
tickers = SPY_info_df.Symbol.to_list()

In [17]:
set([x.replace('-', '.') for x in tickers]) - set(statements.keys())

{'BALL', 'ELV', 'GEHC', 'GEN', 'META', 'TRGP'}

In [34]:
ticker = 'AMZN'
base_url = f"https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=Q&section="
sofp = f"{base_url}Balance%20Sheet&sort=desc"
soci = f"{base_url}Income%20Statement&sort=desc"
socf = f"{base_url}Cash%20Flow&sort=desc"
df1 = pd.read_excel(sofp) # balance sheet data           
df2 = pd.read_excel(soci) # income statement data
df3 = pd.read_excel(socf) # cashflow statement data
df = pd.concat([df1, df2, df3]) # combining all extracted information
df.head()

,Unnamed: 0,2022-09-30 00:00:00,2022-06-30 00:00:00,2022-03-31 00:00:00,2021-12-31 00:00:00,2021-09-30 00:00:00,2021-06-30 00:00:00,2021-03-31 00:00:00,2020-12-31 00:00:00,2020-09-30 00:00:00,...,2015-06-30 00:00:00,2015-03-31 00:00:00,2014-12-31 00:00:00,2014-09-30 00:00:00,2014-06-30 00:00:00,2014-03-31 00:00:00,2013-12-31 00:00:00,2013-09-30 00:00:00,2013-06-30 00:00:00,2013-03-31 00:00:00
0,Cash and Short Term Investments,5.866200e+10,6.071000e+10,6.638500e+10,9.604900e+10,7.898800e+10,8.989400e+10,7.327000e+10,8.439600e+10,6.840200e+10,...,1.400100e+10,1.378100e+10,1.741600e+10,6.883000e+09,7.986000e+09,8.666000e+09,1.244700e+10,7.689000e+09,7.463000e+09,7.895000e+09
1,Receivables,3.615400e+10,3.480400e+10,3.250400e+10,3.289100e+10,2.861000e+10,2.683500e+10,2.428900e+10,2.454200e+10,2.083200e+10,...,4.920000e+09,4.772000e+09,5.612000e+09,4.373000e+09,4.125000e+09,3.945000e+09,4.767000e+09,3.057000e+09,2.861000e+09,2.516000e+09
2,Inventory,3.664700e+10,3.815300e+10,3.498700e+10,3.264000e+10,3.093300e+10,2.411900e+10,2.384900e+10,2.379500e+10,2.373500e+10,...,7.470000e+09,7.369000e+09,8.299000e+09,7.316000e+09,6.644000e+09,6.716000e+09,7.411000e+09,6.068000e+09,5.420000e+09,5.395000e+09
3,Total current assets,1.314630e+11,1.336670e+11,1.338760e+11,1.615800e+11,1.385310e+11,1.408480e+11,1.214080e+11,1.327330e+11,1.129690e+11,...,2.639100e+10,2.592200e+10,3.132700e+10,1.857200e+10,1.875500e+10,1.932700e+10,2.462500e+10,1.733400e+10,1.628500e+10,1.631300e+10
4,"Property, Plant, Equpment (Net)",1.771950e+11,1.737060e+11,1.684680e+11,1.602810e+11,1.471520e+11,1.335020e+11,1.214610e+11,1.131140e+11,9.998100e+10,...,1.947900e+10,1.773600e+10,1.696700e+10,1.570200e+10,1.408900e+10,1.226700e+10,1.094900e+10,9.991000e+09,8.789000e+09,7.674000e+09


In [ ]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
hist = table[1]
# hist['Date'] = pd.to_datetime(hist['Date'])
hist.head()

Date  Added                        Removed  \
            Date Ticker               Security  Ticker   
0  June 21, 2022    KDP       Keurig Dr Pepper  UA/UAA   
1  June 21, 2022     ON       ON Semiconductor    IPGP   
2   June 8, 2022   VICI        Vici Properties    CERN   
3  April 4, 2022    CPT  Camden Property Trust    PBCT   
4  March 2, 2022    MOH      Molina Healthcare    INFO   

                              \
                    Security   
0               Under Armour   
1              IPG Photonics   
2                     Cerner   
3  People's United Financial   
4                 IHS Markit   

                                              Reason  
                                              Reason  
0                   Market capitalization change.[5]  
1                   Market capitalization change.[5]  
2  S&P 500 constituent Oracle Corp. acquired Cern...  
3  S&P 500 constituent M&T Bank Corp. acquired Pe...  
4  S&P 500 constituent S&P Global Inc. acquired I...

In [ ]:
added = hist['Date'].join(hist['Added'])
added['inSPY'] = True
added['Date'] = pd.to_datetime(added['Date'])
added.head()

,Date,Ticker,Security,inSPY
0,2022-06-21,KDP,Keurig Dr Pepper,True
1,2022-06-21,ON,ON Semiconductor,True
2,2022-06-08,VICI,Vici Properties,True
3,2022-04-04,CPT,Camden Property Trust,True
4,2022-03-02,MOH,Molina Healthcare,True


In [ ]:
removed = hist['Date'].join(hist['Removed'])
removed['inSPY'] = False
removed['Date'] = pd.to_datetime(removed['Date'])
removed.head()

,Date,Ticker,Security,inSPY
0,2022-06-21,UA/UAA,Under Armour,False
1,2022-06-21,IPGP,IPG Photonics,False
2,2022-06-08,CERN,Cerner,False
3,2022-04-04,PBCT,People's United Financial,False
4,2022-03-02,INFO,IHS Markit,False


In [ ]:
hist = pd.concat([added, removed]).sort_values('Date', ascending=False)
hist.set_index('Date', inplace=True)
hist.dropna(inplace=True)
hist.head()

,Ticker,Security,inSPY
Date,,,
2022-06-21,KDP,Keurig Dr Pepper,True
2022-06-21,ON,ON Semiconductor,True
2022-06-21,IPGP,IPG Photonics,False
2022-06-21,UA/UAA,Under Armour,False
2022-06-08,CERN,Cerner,False


In [107]:
securities_list = fa.available_companies('a812dc7d35321ed0502ea50d8dc71be9')

In [108]:
len(securities_list)

67339

In [5]:
securities_list['type'].unique()

array(['etf', 'stock', 'trust', 'fund'], dtype=object)

In [6]:
securities_list[securities_list['type'] == 'etf']

,name,price,exchange,exchangeShortName,type
symbol,,,,,
SPY,SPDR S&P 500 ETF Trust,396.91,New York Stock Exchange Arca,AMEX,etf
GDX,VanEck Gold Miners ETF,24.19,New York Stock Exchange Arca,AMEX,etf
EEM,iShares MSCI Emerging Markets ETF,38.25,New York Stock Exchange Arca,AMEX,etf
XLF,Financial Select Sector SPDR Fund,33.74,New York Stock Exchange Arca,AMEX,etf
EFA,iShares MSCI EAFE ETF,60.49,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...
ASOL.SW,21Shares Solana staking ETP,19.08,Swiss Exchange,SIX,etf
AVAX.SW,21Shares Avalanche ETP,3.63,Swiss Exchange,SIX,etf
ABNB.SW,21Shares Binance BNB ETP,20.12,Swiss Exchange,SIX,etf
